In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Prediction_Attrition_Client_Bancaire").getOrCreate()

print("✅ Session Spark créée avec succès")
print("🔹 Version Spark :", spark.version)

✅ Session Spark créée avec succès
🔹 Version Spark : 4.0.1


In [11]:
df = spark.read.csv("../data/dataSet.csv"  , header=True)
df.show()

+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|  Surname|CreditScore|Geography|Gender|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+---------+-----------+---------+------+---+------+---------+-------------+---------+--------------+---------------+------+
|        1|  15634602| Hargrave|        619|   France|Female| 42|     2|        0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|     Hill|        608|    Spain|Female| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|
|        3|  15619304|     Onio|        502|   France|Female| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|
|        4|  15701354|     Boni|        699|   France|Female| 39|     1|        0|            2|        0|             0|       93

In [3]:
df.printSchema()

root
 |-- RowNumber: string (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: string (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Tenure: string (nullable = true)
 |-- Balance: string (nullable = true)
 |-- NumOfProducts: string (nullable = true)
 |-- HasCrCard: string (nullable = true)
 |-- IsActiveMember: string (nullable = true)
 |-- EstimatedSalary: string (nullable = true)
 |-- Exited: string (nullable = true)



In [4]:
df.columns

['RowNumber',
 'CustomerId',
 'Surname',
 'CreditScore',
 'Geography',
 'Gender',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary',
 'Exited']

Étape 3 : Analyse Exploratoire des Données (EDA)

Explorer la structure et la qualité des données :
    Statistiques descriptives globales (.describe().show())
    Identification des valeurs manquantes par colonne via la méthode df.select
    Détection des outliers via la méthode des quantiles approximatifs (approxQuantile) ou IQR
    Analyse groupée avec groupBy() et agrégations (agg, count, avg, sql, etc.)

In [8]:
from pyspark.sql.functions import col
#  il faut corriger les types  : withColumn() → adds or replaces one column at a time.
df = df.withColumn("CreditScore" , col("CreditScore").cast("integer"))
df.show(2)


# changement de colonne en masse 


# Colonnes entières
colonnes_int = ["CreditScore", "Age", "Tenure", "NumOfProducts"]

for c in colonnes_int:
    df = df.withColumn(c, col(c).cast("integer"))

# Colonnes flottantes
colonnes_float = ["Balance", "EstimatedSalary"]

for c in colonnes_float:
    df = df.withColumn(c, col(c).cast("double"))

# Colonnes booléennes (0 ou 1)
colonnes_bool = ["HasCrCard", "IsActiveMember", "Exited"]

for c in colonnes_bool:
    df = df.withColumn(c, col(c).cast("boolean"))

df.printSchema()
df.show(5)


+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId| Surname|CreditScore|Geography|Gender|Age|Tenure| Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
|        1|  15634602|Hargrave|        619|   France|Female| 42|     2|       0|            1|        1|             1|      101348.88|     1|
|        2|  15647311|    Hill|        608|    Spain|Female| 41|     1|83807.86|            1|        0|             1|      112542.58|     0|
+---------+----------+--------+-----------+---------+------+---+------+--------+-------------+---------+--------------+---------------+------+
only showing top 2 rows
root
 |-- RowNumber: string (nullable = true)
 |-- CustomerId: string (nullable = true)
 |-- Surname: string (nullable

In [11]:
#  static descriptivees 
df_desc = df.drop("RowNumber")
df_desc.describe().show()

+-------+-----------------+-------+-----------------+---------+------+------------------+------------------+-----------------+------------------+-----------------+
|summary|       CustomerId|Surname|      CreditScore|Geography|Gender|               Age|            Tenure|          Balance|     NumOfProducts|  EstimatedSalary|
+-------+-----------------+-------+-----------------+---------+------+------------------+------------------+-----------------+------------------+-----------------+
|  count|            10000|  10000|            10000|    10000| 10000|             10000|             10000|            10000|             10000|            10000|
|   mean|  1.56909405694E7|   NULL|         650.5288|     NULL|  NULL|           38.9218|            5.0128|76485.88928799961|            1.5302|100090.2398809998|
| stddev|71936.18612274907|   NULL|96.65329873613035|     NULL|  NULL|10.487806451704587|2.8921743770496837|62397.40520238599|0.5816543579989917|57510.49281769821|
|    min|       

In [14]:
from pyspark.sql import functions as F

df_null = df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns])
df_null.show()


+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|RowNumber|CustomerId|Surname|CreditScore|Geography|Gender|Age|Tenure|Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+
|        0|         0|      0|          0|        0|     0|  0|     0|      0|            0|        0|             0|              0|     0|
+---------+----------+-------+-----------+---------+------+---+------+-------+-------------+---------+--------------+---------------+------+



In [16]:
# detection des outliers 
cols_outliers = [
 'CreditScore',
 'Age',
 'Balance',
 'EstimatedSalary'
 ]

for c in cols_outliers :
    quantiles = df.approxQuantile(c, [0.25, 0.75], 0.05)
    Q1, Q3 = quantiles
    IQR = Q3 - Q1
    borne_inf = Q1 - 1.5 * IQR
    borne_sup = Q3 + 1.5 * IQR
    print(f"{c} → [{borne_inf:.2f}, {borne_sup:.2f}]")


CreditScore → [408.00, 888.00]
Age → [15.50, 59.50]
Balance → [-185753.28, 309588.80]
EstimatedSalary → [-78892.52, 275789.88]


In [21]:
# Analyse groupée avec groupBy() et agrégations (agg, count, avg, sql, etc.)
from pyspark.sql.functions import avg
# nb clt par pays 

df.groupBy("Geography").count().show()
df.groupBy("Age").count().show()
df.groupBy("Gender").count().show()
df.groupBy("IsActiveMember").count().show()

+---------+-----+
|Geography|count|
+---------+-----+
|  Germany| 2509|
|   France| 5014|
|    Spain| 2477|
+---------+-----+

+---+-----+
|Age|count|
+---+-----+
| 31|  404|
| 85|    1|
| 65|   18|
| 53|   74|
| 78|    5|
| 34|  447|
| 81|    4|
| 28|  273|
| 76|   11|
| 27|  209|
| 26|  200|
| 44|  257|
| 22|   84|
| 47|  175|
| 52|  102|
| 40|  432|
| 20|   40|
| 57|   75|
| 54|   84|
| 48|  168|
+---+-----+
only showing top 20 rows
+------+-----+
|Gender|count|
+------+-----+
|Female| 4543|
|  Male| 5457|
+------+-----+

+--------------+-----+
|IsActiveMember|count|
+--------------+-----+
|          true| 5151|
|         false| 4849|
+--------------+-----+



In [20]:
df.groupBy("Gender").agg(avg("Age").alias("Age_Moyen")).show()

+------+------------------+
|Gender|         Age_Moyen|
+------+------------------+
|Female|39.238388729914156|
|  Male|38.658237126626354|
+------+------------------+



In [40]:
# view 
# df.createOrReplaceTempView("client")
# spark.sql("""
#     SELECT Geography, COUNT(*) as NombreClients, AVG(Balance) as Balance_Moyenne
#     FROM clients
#     GROUP BY Geography
# """).show()

## Étape 4 : Prétraitement des Données


Nettoyer et préparer les données pour le machine learning :
    Suppression des colonnes non pertinentes.
    Imputation des valeurs manquantes (Imputer avec stratégie mean, median ou mode)
    Encodage des variables catégorielles (StringIndexer, OneHotEncoder)

In [2]:
# spark.stop()  # Arrête l'ancienne session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Prediction_Attrition_Client_Bancaire") \
    .config("spark.mongodb.output.uri", "mongodb://localhost:27017/clients_bancaires.df_clean") \
    .getOrCreate()

In [3]:
# suppression des colonnes non pertinentes
df = spark.read.csv("../data/dataSet.csv"  , header=True)
df_clean = df.drop("RowNumber")

In [ ]:
# il y a pas d valeurs manquantes dans le dataset

In [4]:
# encodage des variables catégorielles
from pyspark.ml.feature import StringIndexer, OneHotEncoder

index_geo = StringIndexer(inputCol="Geography", outputCol="GeographyIndex")
df_clean= index_geo.fit(df_clean).transform(df_clean)
encoder_geo = OneHotEncoder(inputCol="GeographyIndex", outputCol="GeographyVec")
df_clean = encoder_geo.fit(df_clean).transform(df_clean)


In [5]:
# Encodage : 
df_clean.select("Geography", "GeographyIndex", "GeographyVec").distinct().show()
#  on a un vecteur qui contient 3 valeurs 
# 0 france 1 germany 2 spain , on a supprime spain 
#  (2,[],[]) 
# Le premier nombre → taille du vecteur (ici 2, car 3 catégories − 1 supprimée)
# Le second → indices où la valeur est non nulle
# Le troisième → valeurs correspondantes (souvent 1.0)
#   France| (2,[0],[1.0]) on a vecteur de 2 val , la valeur est postion 0 , true

+---------+--------------+-------------+
|Geography|GeographyIndex| GeographyVec|
+---------+--------------+-------------+
|    Spain|           2.0|    (2,[],[])|
|   France|           0.0|(2,[0],[1.0])|
|  Germany|           1.0|(2,[1],[1.0])|
+---------+--------------+-------------+



In [6]:
index_gender = StringIndexer(inputCol="Gender", outputCol="GenderIndex")
df_clean= index_gender.fit(df_clean).transform(df_clean)
df_clean.select("Gender" , "GenderIndex").distinct().show()


+------+-----------+
|Gender|GenderIndex|
+------+-----------+
|  Male|        0.0|
|Female|        1.0|
+------+-----------+



In [7]:
# <!-- supprission de col intermediare -->
df_clean = df_clean.drop("Geography")
df_clean = df_clean.drop("GeographyIndex")
df_clean = df_clean.drop("Gender")
df_clean.show(5)

+----------+--------+-----------+---+------+---------+-------------+---------+--------------+---------------+------+-------------+-----------+
|CustomerId| Surname|CreditScore|Age|Tenure|  Balance|NumOfProducts|HasCrCard|IsActiveMember|EstimatedSalary|Exited| GeographyVec|GenderIndex|
+----------+--------+-----------+---+------+---------+-------------+---------+--------------+---------------+------+-------------+-----------+
|  15634602|Hargrave|        619| 42|     2|        0|            1|        1|             1|      101348.88|     1|(2,[0],[1.0])|        1.0|
|  15647311|    Hill|        608| 41|     1| 83807.86|            1|        0|             1|      112542.58|     0|    (2,[],[])|        1.0|
|  15619304|    Onio|        502| 42|     8| 159660.8|            3|        1|             0|      113931.57|     1|(2,[0],[1.0])|        1.0|
|  15701354|    Boni|        699| 39|     1|        0|            2|        0|             0|       93826.63|     0|(2,[0],[1.0])|        1.0|

In [8]:
import os
os.environ["HADOOP_HOME"] = r"C:\hadoop"
os.environ["PATH"] = r"C:\hadoop\bin;" + os.environ["PATH"]

In [9]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Prediction_Attrition_Client_Bancaire") \
    .config("spark.hadoop.home.dir", "C:\\hadoop") \
    .getOrCreate()
# UDF pour convertir le vecteur en string
vector_to_string = udf(lambda v: str(v.toArray()) if v else "", StringType())

# Transformer GeographyVec en string et supprimer l’ancienne colonne
df_for_csv = df_clean.withColumn("GeographyVec_str", vector_to_string("GeographyVec")) \
                     .drop("GeographyVec")

# Sauvegarder en CSV
df_for_csv.write.mode("overwrite").option("header", True).csv("C:/Users/elkho/Desktop/df_clean_dataSet.csv")

df_for_csv.show(5)




Py4JJavaError: An error occurred while calling o200.csv.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://cwiki.apache.org/confluence/display/HADOOP2/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:789)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:298)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:314)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:1116)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:798)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:838)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:810)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:837)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(Ra

In [10]:
print(spark.version)

4.0.1


##  Étape 5 : Stockage Intermédiaire dans MongoDB

In [ ]:
! pip install pyspark[mongo]


In [16]:
import pandas as pd
from pyspark.ml.linalg import SparseVector, DenseVector

# Conversion Spark DataFrame -> Pandas DataFrame
df_pandas = df_clean.toPandas()

# Convert SparseVector and DenseVector columns to lists
def convert_vector_to_list(x):
    """Convert PySpark vectors to Python lists"""
    if isinstance(x, (SparseVector, DenseVector)):
        return x.toArray().tolist()
    return x

# Apply conversion to all columns
for col in df_pandas.columns:
    if df_pandas[col].dtype == 'object':
        df_pandas[col] = df_pandas[col].apply(convert_vector_to_list)

# Convert DataFrame to records (list of dictionaries)
records = df_pandas.to_dict('records')

In [17]:
from pymongo import MongoClient

# Connexion à MongoDB
client = MongoClient("mongodb://localhost:27017/") 
db = client["clients_bancaires"]                          
collection = db["df_clean_collection"]

In [19]:
# Insert the data
collection.insert_many(records)

InsertManyResult([ObjectId('690cd88e090bc0bf2e7eb553'), ObjectId('690cd88e090bc0bf2e7eb554'), ObjectId('690cd88e090bc0bf2e7eb555'), ObjectId('690cd88e090bc0bf2e7eb556'), ObjectId('690cd88e090bc0bf2e7eb557'), ObjectId('690cd88e090bc0bf2e7eb558'), ObjectId('690cd88e090bc0bf2e7eb559'), ObjectId('690cd88e090bc0bf2e7eb55a'), ObjectId('690cd88e090bc0bf2e7eb55b'), ObjectId('690cd88e090bc0bf2e7eb55c'), ObjectId('690cd88e090bc0bf2e7eb55d'), ObjectId('690cd88e090bc0bf2e7eb55e'), ObjectId('690cd88e090bc0bf2e7eb55f'), ObjectId('690cd88e090bc0bf2e7eb560'), ObjectId('690cd88e090bc0bf2e7eb561'), ObjectId('690cd88e090bc0bf2e7eb562'), ObjectId('690cd88e090bc0bf2e7eb563'), ObjectId('690cd88e090bc0bf2e7eb564'), ObjectId('690cd88e090bc0bf2e7eb565'), ObjectId('690cd88e090bc0bf2e7eb566'), ObjectId('690cd88e090bc0bf2e7eb567'), ObjectId('690cd88e090bc0bf2e7eb568'), ObjectId('690cd88e090bc0bf2e7eb569'), ObjectId('690cd88e090bc0bf2e7eb56a'), ObjectId('690cd88e090bc0bf2e7eb56b'), ObjectId('690cd88e090bc0bf2e7eb5